<a href="https://colab.research.google.com/github/Jonathanpro/myaiblog/blob/master/_notebooks/2021-07-07-huggingface_multi_class_pytorch_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP -  Multi Class Classification - Training
> A tutorial for Training a binary class classifier with Huggingface & Pytroch.

- toc: true 
- badges: true
- comments: false
- categories: [jupyter]
- image: images/chart-preview.png

# Usecase: Classify titles of news headline to specific categories
### Train model
---


Example:
"Kim Kardashian Flashes Cleavage As She Arrives..." is mapped to "Entertainment"

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.5MB 5.0MB/s 
     |████████████████████████████████| 3.3MB 30.3MB/s 
     |████████████████████████████████| 901kB 36.6MB/s 


Optional for using TPU. Make sure to select TPU from Edit > Notebook settings > Hardware accelerator

In [ ]:
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

     |████████████████████████████████| 149.9MB 76kB/s 
     |████████████████████████████████| 61kB 2.9MB/s 
ERROR: earthengine-api 0.1.269 has requirement google-api-python-client<2,>=1.12.1, but you'll have google-api-python-client 1.8.0 which is incompatible.
  Found existing installation: google-api-python-client 1.12.8
    Uninstalling google-api-python-client-1.12.8:
      Successfully uninstalled google-api-python-client-1.12.8


In [ ]:
# Importing the libraries needed
import pandas as pd
import torch
import transformers
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertModel, DistilBertTokenizer

In [ ]:
# Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

Download public data to use in the notebook

In [ ]:
! wget https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip

--2021-06-29 07:22:42--  https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29224203 (28M) [application/x-httpd-php]
Saving to: ‘NewsAggregatorDataset.zip’

NewsAggregatorDatas 100%[===================>]  27.87M  32.6MB/s    in 0.9s    

2021-06-29 07:22:43 (32.6 MB/s) - ‘NewsAggregatorDataset.zip’ saved [29224203/29224203]



In [ ]:
!unzip NewsAggregatorDataset.zip

Archive:  NewsAggregatorDataset.zip
  inflating: 2pageSessions.csv       
   creating: __MACOSX/
  inflating: __MACOSX/._2pageSessions.csv  
  inflating: newsCorpora.csv         
  inflating: __MACOSX/._newsCorpora.csv  
  inflating: readme.txt              
  inflating: __MACOSX/._readme.txt   


In [ ]:
!ls

2pageSessions.csv  NewsAggregatorDataset.zip  readme.txt
__MACOSX	   newsCorpora.csv	      sample_data


Import the csv into pandas dataframe and add the headers

In [ ]:
df = pd.read_csv('newsCorpora.csv', sep='\t', names=['ID','TITLE', 'URL', 'PUBLISHER', 'CATEGORY', 'STORY', 'HOSTNAME', 'TIMESTAMP'])

Choose a sample based on our available processing power

In [ ]:
# df = df.sample(frac=0.1)

In [ ]:
# Removing unwanted columns and only leaving title of news and the category which will be the target
df = df[['TITLE','CATEGORY']]

# Converting the codes to appropriate categories using a dictionary
my_dict = {
    'e':'Entertainment',
    'b':'Business',
    't':'Science',
    'm':'Health'
}

def update_cat(x):
    return my_dict[x]

df['CATEGORY'] = df['CATEGORY'].apply(lambda x: update_cat(x))

encode_dict = {}

def encode_cat(x):
    if x not in encode_dict.keys():
        encode_dict[x]=len(encode_dict)
    return encode_dict[x]

df['ENCODE_CAT'] = df['CATEGORY'].apply(lambda x: encode_cat(x))

Our actuall data that is used

In [ ]:
df.head()

,TITLE,CATEGORY,ENCODE_CAT
0,"Fed official says weak data caused by weather,...",Business,0
1,Fed's Charles Plosser sees high bar for change...,Business,0
2,US open: Stocks fall after Fed official hints ...,Business,0
3,"Fed risks falling 'behind the curve', Charles ...",Business,0
4,Fed's Plosser: Nasty Weather Has Curbed Job Gr...,Business,0


List of a categories

In [ ]:
labels = list(df['CATEGORY'].unique())

In [ ]:
df_mapping=df[['CATEGORY', 'ENCODE_CAT']].drop_duplicates()
df_mapping = df_mapping.reset_index()
id2label = pd.Series(df_mapping.CATEGORY,index=df_mapping.ENCODE_CAT).to_dict()

In [ ]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 512
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 1
LEARNING_RATE = 1e-05
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased', id2label=id2label)

In [ ]:
class newsCorpora_data(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        title = str(self.data.TITLE[index])
        title = " ".join(title.split())
        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        inputs['label'] = torch.tensor(self.data.ENCODE_CAT[index], dtype=torch.long)
        del inputs['token_type_ids']
        return inputs
        """
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'label': torch.tensor(self.data.ENCODE_CAT[index], dtype=torch.long)
        } 
        """
    def __len__(self):
        return self.len

In [ ]:
# Creating the dataset and dataloader for the neural network

train_size = 0.8
train_dataset = df.sample(frac=train_size,random_state=200)
test_dataset = df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)


training_set = newsCorpora_data(train_dataset, tokenizer, MAX_LEN)
testing_set = newsCorpora_data(test_dataset, tokenizer, MAX_LEN)

Set device to TPU

In [ ]:
import torch_xla
import torch_xla.core.xla_model as xm

device = xm.xla_device()

In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=1*16,  # batch size per device during training
    per_device_eval_batch_size=1*64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=5000,
   save_strategy='steps', save_steps=5000 
)
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-cased", num_labels=len(labels), id2label=id2label)
# Map model to TPU
model = model.to(device)
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=training_set,         # training dataset
    eval_dataset=testing_set             # evaluation dataset
)



Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'classifier.weigh

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 337935
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 63363
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2132: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Step,Training Loss
5000,0.347500
10000,0.224900
15000,0.203000
20000,0.190500
25000,0.144800
30000,0.135700
35000,0.131100
40000,0.128900
45000,0.104000
50000,0.085600


Saving model checkpoint to ./results/checkpoint-5000
Configuration saved in ./results/checkpoint-5000/config.json
Model weights saved in ./results/checkpoint-5000/pytorch_model.bin
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2132: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Saving model checkpoint to ./results/checkpoint-10000
Configuration saved in ./results/checkpoint-10000/config.json
Model weights saved in ./results/checkpoint-10000/pytorch_model.bin
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2132: FutureWarning: The `pad

TrainOutput(global_step=63363, training_loss=0.15106624429101714, metrics={'train_runtime': 16966.2362, 'train_samples_per_second': 59.754, 'train_steps_per_second': 3.735, 'total_flos': 2.048800853818368e+17, 'train_loss': 0.15106624429101714, 'epoch': 3.0})

Save model to google drive for later use(evaluation & inference)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
trainer.save_model("/content/gdrive/MyDrive/MC_01_03_29_06_2021.bin")

Saving model checkpoint to /content/gdrive/MyDrive/MC_01_03_29_06_2021.bin
Configuration saved in /content/gdrive/MyDrive/MC_01_03_29_06_2021.bin/config.json
Model weights saved in /content/gdrive/MyDrive/MC_01_03_29_06_2021.bin/pytorch_model.bin
